In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import pandas as pd
import numpy as np
import cv2
from keras.utils import np_utils
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D,Activation,MaxPooling2D
from keras.utils import normalize
from keras.layers import Concatenate
from keras import Input
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle"

In [ ]:
%cd /content/gdrive/MyDrive/Kaggle

/content/gdrive/MyDrive/Kaggle


In [ ]:
%ls

kaggle.json


In [ ]:
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database

 99% 740M/745M [00:06<00:00, 72.2MB/s]
100% 745M/745M [00:06<00:00, 114MB/s] 


In [ ]:
!unzip \*.zip  && rm *.zip

Streaming output truncated to the last 5000 lines.
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-6710.png  
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-6711.png  
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-6712.png  
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-6713.png  
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-6714.png  
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-6715.png  
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-6716.png  
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-6717.png  
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-6718.png  
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-6719.png  
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-672.png  
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-6720.png  
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-6721.png  
  inflating: COVID-19_Radiography_Dataset/Normal/Normal-6722.png  
  inflating:

In [ ]:
%cd /content/gdrive/MyDrive/Kaggle/COVID-19_Radiography_Dataset

/content/gdrive/MyDrive/Kaggle/COVID-19_Radiography_Dataset


In [ ]:
categories = ['Normal', 'COVID']
labels = [0, 1]
dictionnaire = {'Normal': 0, 'COVID': 1}

In [ ]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=category
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)     
            resized=cv2.resize(gray,(img_size,img_size))
            data.append(resized)
            print("data size in ",category," is : ",len(data))
            target.append(dictionnaire[category])

        except Exception as e:
            print('Exception:',e)

Streaming output truncated to the last 5000 lines.
data size in  Normal  is :  8809
data size in  Normal  is :  8810
data size in  Normal  is :  8811
data size in  Normal  is :  8812
data size in  Normal  is :  8813
data size in  Normal  is :  8814
data size in  Normal  is :  8815
data size in  Normal  is :  8816
data size in  Normal  is :  8817
data size in  Normal  is :  8818
data size in  Normal  is :  8819
data size in  Normal  is :  8820
data size in  Normal  is :  8821
data size in  Normal  is :  8822
data size in  Normal  is :  8823
data size in  Normal  is :  8824
data size in  Normal  is :  8825
data size in  Normal  is :  8826
data size in  Normal  is :  8827
data size in  Normal  is :  8828
data size in  Normal  is :  8829
data size in  Normal  is :  8830
data size in  Normal  is :  8831
data size in  Normal  is :  8832
data size in  Normal  is :  8833
data size in  Normal  is :  8834
data size in  Normal  is :  8835
data size in  Normal  is :  8836
data size in  Normal  is 

In [ ]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)
new_target=np_utils.to_categorical(target)

In [ ]:
np.save('data',data)
np.save('target',new_target)

In [ ]:
data=np.load('data.npy')
target=np.load('target.npy')

In [ ]:
input_shape=data.shape[1:]
inp=Input(shape=input_shape)
convs=[]

parrallel_kernels=[3,5,7]

for k in range(len(parrallel_kernels)):

    conv = Conv2D(128, parrallel_kernels[k],padding='same',activation='relu',input_shape=input_shape,strides=1)(inp)
    convs.append(conv)

out = Concatenate()(convs)
conv_model = Model(inp,out)

model = Sequential()
model.add(conv_model)

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,input_dim=128,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#Informations sur le modele CNN
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 100, 100, 384)     11008     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 98, 98, 64)        221248    
_________________________________________________________________
activation (Activation)      (None, 98, 98, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 47, 32)        18464     
_________________________________________________________________
activation_1 (Activation)    (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0

In [ ]:
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [ ]:
from keras.models import load_model
model = load_model('/content/gdrive/MyDrive/Kaggle/COVID-19_Radiography_Dataset/model-003.model')

In [ ]:
loss, acc = model.evaluate(test_data, test_target, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

44/44 - 121s - loss: 0.6157 - accuracy: 0.7451
Untrained model, accuracy: 74.51%


In [ ]:
model.save_weights('testmodelwieghts.model')

In [ ]:
%cd /content/gdrive/MyDrive/Kaggle/COVID-19_Radiography_Dataset

/content/gdrive/MyDrive/Kaggle/COVID-19_Radiography_Dataset


In [ ]:
checkpoint_filepath = '/Checkpoints'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint,model_checkpoint_callback],validation_split=0.1)

Epoch 1/20
350/350 [==============================] - 4047s 12s/step - loss: 0.1708 - accuracy: 0.9353 - val_loss: 0.1308 - val_accuracy: 0.9493
INFO:tensorflow:Assets written to: model-001.model/assets
Epoch 2/20
350/350 [==============================] - 4051s 12s/step - loss: 0.1534 - accuracy: 0.9457 - val_loss: 0.1010 - val_accuracy: 0.9686
INFO:tensorflow:Assets written to: model-002.model/assets
Epoch 3/20
350/350 [==============================] - 4053s 12s/step - loss: 0.1280 - accuracy: 0.9534 - val_loss: 0.1102 - val_accuracy: 0.9566
Epoch 4/20
350/350 [==============================] - 4036s 12s/step - loss: 0.1190 - accuracy: 0.9561 - val_loss: 0.0993 - val_accuracy: 0.9646
INFO:tensorflow:Assets written to: model-004.model/assets
Epoch 5/20
350/350 [==============================] - 4032s 12s/step - loss: 0.1063 - accuracy: 0.9634 - val_loss: 0.1008 - val_accuracy: 0.9678
Epoch 6/20
350/350 [==============================] - 4013s 11s/step - loss: 0.0990 - accuracy: 0.964

gdfgdfg